In [1]:
from pathlib import Path

import torch
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from neuralhydrology.modelzoo.cudalstm import CudaLSTM
from neuralhydrology.utils.config import Config
from neuralhydrology.datasetzoo import get_dataset
from neuralhydrology.datautils.utils import load_scaler

In [2]:
BASIN_ID = 11001
EPOCH = 1

In [3]:
run_dir = Path("./runs/lstm_2407_174255")
config = Config(run_dir / "config.yml")
config.as_dict()

{'batch_size': 512,
 'clip_gradient_norm': 1,
 'commit_hash': '7806a39',
 'data_dir': PosixPath('../data/CAMELS_US'),
 'dataset': 'camels_us',
 'device': 'cuda:0',
 'dynamic_inputs': ['prcp(mm/day)',
  'srad(W/m2)',
  'tmax(C)',
  'tmin(C)',
  'vp(Pa)'],
 'epochs': 1,
 'experiment_name': 'lstm',
 'forcings': ['daymet'],
 'head': 'regression',
 'hidden_size': 256,
 'img_log_dir': PosixPath('/home/spectre/Projects/ISSAI/Internship/aqua_rate/ML/runs/lstm_2407_174255/img_log'),
 'initial_forget_bias': 3,
 'learning_rate': {0: 0.01, 10: 0.001, 30: 0.0001, 40: 1e-05},
 'log_interval': 5,
 'log_tensorboard': True,
 'loss': 'NSE',
 'metrics': ['NSE', 'KGE'],
 'model': 'cudalstm',
 'num_workers': 8,
 'number_of_basins': 531,
 'optimizer': 'Adam',
 'output_activation': 'linear',
 'output_dropout': 0.4,
 'package_version': '1.10.0',
 'predict_last_n': 1,
 'run_dir': PosixPath('/home/spectre/Projects/ISSAI/Internship/aqua_rate/ML/runs/lstm_2407_174255'),
 'save_validation_results': True,
 'save_we

In [4]:
model = CudaLSTM(cfg=config)
model = model.eval()
model

CudaLSTM(
  (embedding_net): InputLayer(
    (statics_embedding): Identity()
    (dynamics_embedding): Identity()
  )
  (lstm): LSTM(31, 256)
  (dropout): Dropout(p=0.4, inplace=False)
  (head): Regression(
    (net): Sequential(
      (0): Linear(in_features=256, out_features=1, bias=True)
    )
  )
)

In [5]:
model.load_state_dict(torch.load(run_dir / f"model_epoch00{EPOCH}.pt"))
model

CudaLSTM(
  (embedding_net): InputLayer(
    (statics_embedding): Identity()
    (dynamics_embedding): Identity()
  )
  (lstm): LSTM(31, 256)
  (dropout): Dropout(p=0.4, inplace=False)
  (head): Regression(
    (net): Sequential(
      (0): Linear(in_features=256, out_features=1, bias=True)
    )
  )
)

In [6]:
# ds = get_dataset(cfg=config, is_train=False, basin=str(BASIN_ID), period="test", scaler=load_scaler(run_dir))
ds = get_dataset(cfg=config, is_train=False, basin=str("01350000"), period="validation", scaler=load_scaler(run_dir))
loader = DataLoader(ds, batch_size=1, num_workers=0, collate_fn=ds.collate_fn)

In [7]:
for key, value in next(iter(loader)).items():
    print(f"{key}: {value.shape}")

x_d: torch.Size([1, 365, 5])
y: torch.Size([1, 365, 1])
date: (1, 365)
x_s: torch.Size([1, 26])
per_basin_target_stds: torch.Size([1, 1, 1])


In [8]:
next(iter(loader))

{'x_d': tensor([[[        nan,         nan,         nan,         nan,         nan],
          [        nan,         nan,         nan,         nan,         nan],
          [        nan,         nan,         nan,         nan,         nan],
          ...,
          [-4.1685e-01,  3.8748e-01, -8.6978e-02, -4.5249e-01, -5.5161e-01],
          [-4.1685e-01, -2.8148e-01,  4.6476e-02,  2.1517e-01,  1.4025e-04],
          [-4.1685e-01,  1.0271e-01,  3.3989e-01,  1.8903e-01, -3.1505e-02]]]),
 'y': tensor([[[    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
    

In [13]:
for i, data in enumerate(loader):
    if i == 3 * 31 - 3:
        break

data

{'x_d': tensor([[[-0.4169, -1.0465, -1.4051, -1.2056, -0.9394],
          [-0.4169, -0.9556, -1.4279, -1.3062, -0.9955],
          [-0.4169, -0.8831, -1.5888, -1.5395, -1.0712],
          ...,
          [-0.4169, -0.7563, -1.6327, -1.9909, -1.1935],
          [-0.4169, -1.3819, -1.2899, -0.9472, -0.8307],
          [-0.4169, -0.9556, -1.0989, -1.1544, -0.9312]]]),
 'y': tensor([[[    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan]

In [ ]:
model.embedding_net(next(iter(loader))), model.embedding_net(next(iter(loader))).shape

(tensor([[[-0.4169, -0.5676,  0.5913,  ...,  0.4426,  0.7623, -1.2553]],
 
         [[-0.1162, -0.7215,  0.3234,  ...,  0.4426,  0.7623, -1.2553]],
 
         [[-0.4169, -0.7431, -0.1647,  ...,  0.4426,  0.7623, -1.2553]],
 
         ...,
 
         [[-0.4169, -0.1123, -0.0175,  ...,  0.4426,  0.7623, -1.2553]],
 
         [[-0.4169,  0.0936,  0.0611,  ...,  0.4426,  0.7623, -1.2553]],
 
         [[-0.4169,  0.1456, -0.0266,  ...,  0.4426,  0.7623, -1.2553]]]),
 torch.Size([365, 1, 31]))

In [ ]:
input_data = next(iter(loader))
# data preprocess
input_data = model.pre_model_hook(input_data, is_train=False)
# forward pass
pred = model(input_data)
pred

{'lstm_output': tensor([[[-5.9320e-02, -1.3002e-01,  7.8301e-02,  ..., -1.2561e-03,
            1.3747e-01, -1.4410e-02],
          [-2.5383e-02, -8.6849e-03,  5.1158e-04,  ..., -3.4279e-05,
            8.8796e-03, -6.0736e-04],
          [-3.2710e-03, -4.4730e-04,  9.1990e-06,  ..., -6.0686e-07,
            3.9548e-03, -4.0607e-05],
          ...,
          [-1.2772e-07, -4.8127e-05,  2.7990e-07,  ...,  4.3472e-08,
           -3.5956e-03,  1.6068e-09],
          [-3.8727e-08, -2.5508e-05,  1.7765e-07,  ...,  2.1678e-08,
           -3.4819e-03,  6.7666e-10],
          [-3.4195e-08, -1.9193e-05,  2.0943e-07,  ...,  2.6699e-08,
           -3.5238e-03,  4.6906e-10]]], grad_fn=<TransposeBackward0>),
 'h_n': tensor([[[-3.4195e-08, -1.9193e-05,  2.0943e-07,  3.1109e-04, -2.2332e-04,
           -9.4615e-04, -1.1377e-05,  3.0866e-10,  9.9093e-13,  9.9973e-01,
            9.6745e-08,  2.4300e-05,  8.3828e-05, -4.3573e-04,  1.2764e-04,
           -2.7054e-02,  1.0128e-07,  9.1194e-05,  3.3396e-0

In [ ]:
torch.cat([input_data["x_d"], input_data["x_s"].unsqueeze(0).expand(1, 365, 26)], dim=2)

tensor([[[-0.4169, -0.5676,  0.5913,  ...,  0.4426,  0.7623, -1.2553],
         [-0.1162, -0.7215,  0.3234,  ...,  0.4426,  0.7623, -1.2553],
         [-0.4169, -0.7431, -0.1647,  ...,  0.4426,  0.7623, -1.2553],
         ...,
         [-0.4169, -0.1123, -0.0175,  ...,  0.4426,  0.7623, -1.2553],
         [-0.4169,  0.0936,  0.0611,  ...,  0.4426,  0.7623, -1.2553],
         [-0.4169,  0.1456, -0.0266,  ...,  0.4426,  0.7623, -1.2553]]])

In [ ]:
torch.cat([input_data["x_d"], input_data["x_s"].unsqueeze(0).expand(1, 365, 26)], dim=2).shape

torch.Size([1, 365, 31])

In [ ]:
for key, value in pred.items():
    print(f"{key}: {value.shape}")

lstm_output: torch.Size([1, 365, 256])
h_n: torch.Size([1, 1, 256])
c_n: torch.Size([1, 1, 256])
y_hat: torch.Size([1, 365, 1])


In [ ]:
pred["lstm_output"][0][-1]

tensor([ 0.2017,  0.8636,  0.2958,  0.1563,  0.0373, -0.0397, -0.0239,  0.7562,
         0.5485, -0.2235, -0.3931,  0.0771,  0.7660, -0.7195,  0.1938,  0.2186,
        -0.4614,  0.0335,  0.0389,  0.4567, -0.1343, -0.7029, -0.9307,  0.9442,
        -0.9445, -0.1452,  0.2349,  0.5704,  0.1872, -0.1445, -0.0234, -0.4239,
        -0.2382,  0.7324,  0.7778,  0.9067, -0.9841,  0.5849,  0.1926, -0.9700,
        -0.0079, -0.5936, -0.6096, -0.3110,  0.0975,  0.9417,  0.2364,  0.6696,
         0.7428,  0.0267,  0.2411,  0.0682,  0.7617, -0.3002,  0.9063, -0.6722,
         0.0125,  0.1069,  0.6384,  0.9328,  0.4389,  0.9762, -0.9865,  0.0203,
        -0.0437,  0.0324,  0.4318,  0.0319,  0.0421,  0.3862, -0.6914, -0.7804,
         0.0204, -0.7616,  0.1187, -0.0811, -0.1895,  0.0911, -0.0249, -0.2892,
        -0.9131,  0.0303,  0.3639, -0.8298, -0.3709, -0.0777, -0.0676,  0.9462,
         0.8544,  0.1344,  0.1919, -0.5161, -0.0378, -0.7810, -0.9624,  0.2910,
         0.1978, -0.0643, -0.5471,  0.53

In [ ]:
pred["h_n"][0][0]

tensor([ 0.2017,  0.8636,  0.2958,  0.1563,  0.0373, -0.0397, -0.0239,  0.7562,
         0.5485, -0.2235, -0.3931,  0.0771,  0.7660, -0.7195,  0.1938,  0.2186,
        -0.4614,  0.0335,  0.0389,  0.4567, -0.1343, -0.7029, -0.9307,  0.9442,
        -0.9445, -0.1452,  0.2349,  0.5704,  0.1872, -0.1445, -0.0234, -0.4239,
        -0.2382,  0.7324,  0.7778,  0.9067, -0.9841,  0.5849,  0.1926, -0.9700,
        -0.0079, -0.5936, -0.6096, -0.3110,  0.0975,  0.9417,  0.2364,  0.6696,
         0.7428,  0.0267,  0.2411,  0.0682,  0.7617, -0.3002,  0.9063, -0.6722,
         0.0125,  0.1069,  0.6384,  0.9328,  0.4389,  0.9762, -0.9865,  0.0203,
        -0.0437,  0.0324,  0.4318,  0.0319,  0.0421,  0.3862, -0.6914, -0.7804,
         0.0204, -0.7616,  0.1187, -0.0811, -0.1895,  0.0911, -0.0249, -0.2892,
        -0.9131,  0.0303,  0.3639, -0.8298, -0.3709, -0.0777, -0.0676,  0.9462,
         0.8544,  0.1344,  0.1919, -0.5161, -0.0378, -0.7810, -0.9624,  0.2910,
         0.1978, -0.0643, -0.5471,  0.53

In [ ]:
ds._per_basin_target_stds

{'11001': tensor([[0.2768]]),
 '11068': tensor([[0.2847]]),
 '11126': tensor([[1.1639]]),
 '11129': tensor([[1.7891]]),
 '11163': tensor([[nan]]),
 '11164': tensor([[1.7698]]),
 '11275': tensor([[0.2989]]),
 '11293': tensor([[0.1148]]),
 '11395': tensor([[0.7345]]),
 '11397': tensor([[0.0529]]),
 '11421': tensor([[31.4752]]),
 '11433': tensor([[0.9503]]),
 '11469': tensor([[0.5810]]),
 '12002': tensor([[0.0804]]),
 '12008': tensor([[0.0070]]),
 '12032': tensor([[0.1136]]),
 '12072': tensor([[0.0450]]),
 '12075': tensor([[0.0667]]),
 '12564': tensor([[0.1507]]),
 '13002': tensor([[0.0118]]),
 '13005': tensor([[0.2477]]),
 '13038': tensor([[0.0009]]),
 '13048': tensor([[0.1841]]),
 '13064': tensor([[0.1512]]),
 '13090': tensor([[0.2235]]),
 '13091': tensor([[0.3406]]),
 '13115': tensor([[0.0473]]),
 '13128': tensor([[0.1171]]),
 '13221': tensor([[0.1861]]),
 '19022': tensor([[0.0144]]),
 '19195': tensor([[0.0735]]),
 '19196': tensor([[0.0385]]),
 '19205': tensor([[0.1231]]),
 '19208': te

In [ ]:
stds = set()
for data in loader:
    stds.add(data["per_basin_target_stds"][0][0][0].item())

In [ ]:
stds

{0.2767565846443176,
 0.28465375304222107,
 1.1639032363891602,
 1.7891430854797363,
 nan,
 nan,
 1.769752860069275,
 nan,
 0.29886099696159363,
 nan,
 0.11484294384717941,
 nan,
 0.7344523668289185,
 nan,
 0.05291290581226349,
 nan,
 0.04498513415455818,
 nan,
 0.014383990317583084,
 nan,
 0.03850247338414192,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 31.475189208984375,
 nan,
 0.9503335952758789,
 nan,
 0.580983579158783,
 nan,
 0.08041262626647949,
 nan,
 0.007003425620496273,
 nan,
 nan,
 nan,
 0.22350552678108215,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.3406287729740143,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.024880042299628258,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [ ]:
len(stds)

1137

In [ ]:
len(loader)

46032

In [ ]:
ds_basin = get_dataset(cfg=config, is_train=False, basin=str(BASIN_ID), period="train", scaler=load_scaler(run_dir))
loader_basin = DataLoader(ds_basin, batch_size=1, num_workers=0, collate_fn=ds.collate_fn)

In [ ]:
a = next(iter(loader_basin))
a

{'x_d': tensor([[[    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          ...,
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [ 1.0405, -0.8439, -1.4316,  ...,  1.1256, -2.0378,     nan]]]),
 'y': tensor([[[    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [    nan],
          [  

In [ ]:
a["x_d"][0][-1]

tensor([ 1.0405, -0.8439, -1.4316, -2.0470, -1.2082, -0.8746,  1.1256, -2.0378,
            nan])